In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    X = data.drop(columns=['Eligible'])
    y = data['Eligible']
    return X, y

def train_svr_model(X_train, y_train):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    svr_model = SVR()
    svr_model.fit(X_train_scaled, y_train)
    return svr_model, scaler

# Load and train on VehicleTrainingDataset.csv
X_train, y_train = load_and_preprocess_data('VehicleTrainingDataset_Noisy_0.1.csv')
svr_model, scaler = train_svr_model(X_train, y_train)

# Predict eligibility scores on 1000VehicleDataset.csv
vehicles_df = pd.read_csv('1000VehicleDataset_Noisy_0.1.csv')
X_test = vehicles_df.drop(columns=['Eligible'])
X_test_scaled = scaler.transform(X_test)
predicted_scores = svr_model.predict(X_test_scaled)

# Assuming you have access to actual scores, replace this line with the actual score loading logic if available
y_actual = vehicles_df['Eligible']  # This would be prior to overwriting with predictions if you run this block again

# Replace actual scores with predicted ones
vehicles_df['Eligible'] = predicted_scores  

# Calculate metrics
mae = mean_absolute_error(y_actual, predicted_scores)
rmse = np.sqrt(mean_squared_error(y_actual, predicted_scores))
r_squared = r2_score(y_actual, predicted_scores)
rae = np.sum(np.abs(y_actual - predicted_scores)) / np.sum(np.abs(y_actual - np.mean(y_actual)))

# Output the results
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r_squared}")
print(f"RAE: {rae}")


MAE: 1.403494699070916
RMSE: 1.9135086312664382
R-squared: 0.9752350615552381
RAE: 0.14658131997050208


In [2]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback

# Load datasets
tasks_df = pd.read_csv('RandomTasks200.csv')


# Rename columns for consistency
tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)

# Define the Gym environment for task allocation
class TaskAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, vehicles, tasks):
        super(TaskAllocationEnv, self).__init__()
        self.vehicles = vehicles
        self.tasks = tasks
        self.current_task = 0
        self.successful_assignments = 0
        self.successful_history = []  # Added to track successful assignments
        self.seed()

        num_features = tasks.shape[1]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)
        self.eligible_vehicle_indices = []
        self.update_action_space()

    def seed(self, seed=42):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]        
        
    def update_action_space(self):
        task_eligibility = self.tasks.iloc[self.current_task]['MinEligibility']
        eligible_vehicles = self.vehicles[self.vehicles['Eligible'] >= task_eligibility]
        self.eligible_vehicle_indices = eligible_vehicles.index.tolist()
        if len(self.eligible_vehicle_indices) == 0:
            self.action_space = spaces.Discrete(1)  # Prevents invalid action space of size 0
        else:
            self.action_space = spaces.Discrete(len(self.eligible_vehicle_indices))

    def reset(self):
        self.current_task = 0
        self.successful_assignments = 0
        self.update_action_space()
        #print("Resetting environment. Starting new episode.")
        return self.tasks.iloc[self.current_task].values.astype(np.float32)

    def step(self, action):
        task = self.tasks.iloc[self.current_task]
        vehicle = self.vehicles.iloc[action]
        reward = 0

        # Check if the vehicle meets all the task requirements
        meets_requirements = (
            vehicle['RAM'] >= task['RAM'] and
            vehicle['storage'] >= task['storage'] and
            vehicle['Trustfactor'] >= task['Trustfactor'] and
            vehicle['Distance'] <= task['Distance'] and
            vehicle['TransmissionRate'] >= task['TransmissionRate']
        )
        reward = 1 if meets_requirements else -1

        self.successful_assignments += reward > 0
        self.current_task += 1
        done = self.current_task >= len(self.tasks)

        if not done:
            self.update_action_space()
            next_state = self.tasks.iloc[self.current_task].values.astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape[0])
            self.successful_history.append(self.successful_assignments)
            #print(f"Episode completed. Successful assignments: {self.successful_assignments}.")
            self.successful_assignments = 0  # Reset for next episode

        
        
        #three lines below can be uncommented for more detailed output
        #print(f"Task Details: {task.to_dict()}")
        #print(f"Vehicle Details: {vehicle.to_dict()}")
        #print(f"Step: Task {self.current_task}, Action {action}, Reward {reward}")

        return next_state, reward, done, {}

    def get_average_success(self):
        return np.mean(self.successful_history) if self.successful_history else 0


    def render(self, mode='human'):
        pass

    def close(self):
        pass



# Custom callback for logging
class CustomCallback(BaseCallback):
    def __init__(self, env, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.env = env
        self.total_rewards = 0
        self.total_assignments = 0
        self.num_episodes = 0

    def _on_step(self):
        return True

    def _on_rollout_end(self):
        mean_reward, std_reward = evaluate_policy(self.model, self.model.get_env(), n_eval_episodes=10)
        average_assignments = self.env.get_attr('get_average_success')[0]()
        self.total_rewards += mean_reward
        self.total_assignments += average_assignments
        self.num_episodes += 1

        print("-------- Rollout Summary --------")
        print(f"Total mean reward: {mean_reward}")
        print(f"Standard deviation of reward: {std_reward}")
        print(f"Average successful assignments: {average_assignments}")
        print("All assignments history:", self.env.envs[0].successful_history)
        self.env.envs[0].successful_history = []  # Reset history after each iteration

    def _on_training_end(self):
        average_total_reward = self.total_rewards / self.num_episodes
        average_total_assignments = self.total_assignments / self.num_episodes
        print("-------- Training Summary --------")
        print(f"Overall Average Mean Reward: {average_total_reward}")
        print(f"Overall Average Successful Assignments: {average_total_assignments}")


# Prepare the environment
env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, tasks_df), n_envs=1)

# Initialize and train the PPO model
model = PPO("MlpPolicy", env, verbose=1,
            n_steps=1024, batch_size=128, n_epochs=10, learning_rate=0.00018,
            gamma=0.96, gae_lambda=0.87, clip_range=0.15, ent_coef=0.07)

callback = CustomCallback(env)  # Use custom callback for detailed tracking and logging

# Train the model with the custom callback
model.learn(total_timesteps=1024*100, callback=callback)

# Save the model
model.save("ppo_task_allocation_model")


C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
-------- Rollout Summary --------
Total mean reward: -182.0
Standard deviation of reward: 0.0
Average successful assignments: 8.866666666666667
All assignments history: [8, 11, 8, 9, 7, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -183     |
| time/              |          |
|    fps             | 187      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 1024     |
---------------------------------


C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_average_success to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_average_success` for environment variables or `env.get_wrapper_attr('get_average_success')` that will search the reminding wrappers.
  logger.warn(
C:\Users\asdra\anaconda3\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.successful_history to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.successful_history` for environment variables or `env.get_wrapper_attr('successful_history')` that will search the reminding wrappers.
  logger.warn(


-------- Rollout Summary --------
Total mean reward: -132.0
Standard deviation of reward: 0.0
Average successful assignments: 16.733333333333334
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -186        |
| time/                   |             |
|    fps                  | 166         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.007873634 |
|    clip_fraction        | 0.0728      |
|    clip_range           | 0.15        |
|    entropy_loss         | -6.51       |
|    explained_variance   | -0.365      |
|    learning_rate        | 0.00018     |
|    loss                 | 2.29        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0457     |
|    value_loss           | 1

-------- Rollout Summary --------
Total mean reward: -96.0
Standard deviation of reward: 0.0
Average successful assignments: 22.98
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 200          |
|    ep_rew_mean          | -186         |
| time/                   |              |
|    fps                  | 163          |
|    iterations           | 10           |
|    time_elapsed         | 62           |
|    total_timesteps      | 10240        |
| train/                  |              |
|    approx_kl            | 0.0110326735 |
|    clip_fraction        | 0.2          |
|    clip_range           | 0.15         |
|    entropy_loss         | -6.47        |
|    explained_variance   | 0.094        |
|    learning_rate        | 0.00018      |
|    loss                 | 1.57         |
|    n_updates            | 90           |
|    policy_gradient_loss | -0.0451      |
|    value_loss         

...

-------- Rollout Summary --------
Total mean reward: 80.0
Standard deviation of reward: 0.0
Average successful assignments: 73.52727272727273
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 200          |
|    ep_rew_mean          | -98.9        |
| time/                   |              |
|    fps                  | 264          |
|    iterations           | 66           |
|    time_elapsed         | 255          |
|    total_timesteps      | 67584        |
| train/                  |              |
|    approx_kl            | 0.0100050885 |
|    clip_fraction        | 0.219        |
|    clip_range           | 0.15         |
|    entropy_loss         | -4.77        |
|    explained_variance   | 0.624        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.549        |
|    n_updates            | 650          |
|    policy_gradient_loss | -0.0564      |
|    value_lo

-------- Rollout Summary --------
Total mean reward: 76.0
Standard deviation of reward: 0.0
Average successful assignments: 78.13063063063063
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -75.6       |
| time/                   |             |
|    fps                  | 274         |
|    iterations           | 74          |
|    time_elapsed         | 276         |
|    total_timesteps      | 75776       |
| train/                  |             |
|    approx_kl            | 0.010511536 |
|    clip_fraction        | 0.21        |
|    clip_range           | 0.15        |
|    entropy_loss         | -4.41       |
|    explained_variance   | 0.606       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.849       |
|    n_updates            | 730         |
|    policy_gradient_loss | -0.0532     |
|    value_loss           | 2.18

-------- Rollout Summary --------
Total mean reward: 80.0
Standard deviation of reward: 0.0
Average successful assignments: 82.2349593495935
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -52.9       |
| time/                   |             |
|    fps                  | 281         |
|    iterations           | 82          |
|    time_elapsed         | 297         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.011431148 |
|    clip_fraction        | 0.241       |
|    clip_range           | 0.15        |
|    entropy_loss         | -4.22       |
|    explained_variance   | 0.498       |
|    learning_rate        | 0.00018     |
|    loss                 | 0.748       |
|    n_updates            | 810         |
|    policy_gradient_loss | -0.0578     |
|    value_loss           | 2.32 

-------- Rollout Summary --------
Total mean reward: 88.0
Standard deviation of reward: 0.0
Average successful assignments: 86.00888888888889
All assignments history: []
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -35.2       |
| time/                   |             |
|    fps                  | 288         |
|    iterations           | 90          |
|    time_elapsed         | 319         |
|    total_timesteps      | 92160       |
| train/                  |             |
|    approx_kl            | 0.010710815 |
|    clip_fraction        | 0.224       |
|    clip_range           | 0.15        |
|    entropy_loss         | -4          |
|    explained_variance   | 0.49        |
|    learning_rate        | 0.00018     |
|    loss                 | 1.08        |
|    n_updates            | 890         |
|    policy_gradient_loss | -0.0533     |
|    value_loss           | 2.4 

-------- Rollout Summary --------
Total mean reward: 90.0
Standard deviation of reward: 0.0
Average successful assignments: 89.33469387755102
All assignments history: []
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 200          |
|    ep_rew_mean          | -23.4        |
| time/                   |              |
|    fps                  | 294          |
|    iterations           | 98           |
|    time_elapsed         | 340          |
|    total_timesteps      | 100352       |
| train/                  |              |
|    approx_kl            | 0.0091792215 |
|    clip_fraction        | 0.199        |
|    clip_range           | 0.15         |
|    entropy_loss         | -3.91        |
|    explained_variance   | 0.424        |
|    learning_rate        | 0.00018      |
|    loss                 | 0.97         |
|    n_updates            | 970          |
|    policy_gradient_loss | -0.0497      |
|    value_lo

In [3]:
# Load new task dataset for testing
new_tasks_df = pd.read_csv('RandomTasks200Test.csv')
new_tasks_df.rename(columns={
    'Required_RAM': 'RAM',
    'Required_Storage': 'storage',
    'Minimum_Trust_Factor': 'Trustfactor',
    'Max_Distance': 'Distance',
    'Min_Transmission_Rate': 'TransmissionRate',
    'Min_Eligibility': 'MinEligibility'
}, inplace=True)
test_env = make_vec_env(lambda: TaskAllocationEnv(vehicles_df, new_tasks_df), n_envs=1)

# Evaluate the model on the new test environment
mean_reward, std_reward = evaluate_policy(model, test_env, n_eval_episodes=10)
# Extract the successful assignments history from the test environment
successful_assignments = test_env.envs[0].env.get_average_success()

print("---- Testing Summary ----")
print(f"Mean Reward: {mean_reward}")
print(f"Standard Deviation of Reward: {std_reward}")
print(f"Average Successful Assignments in Testing: {successful_assignments}")

C:\Users\asdra\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---- Testing Summary ----
Mean Reward: 38.0
Standard Deviation of Reward: 0.0
Average Successful Assignments in Testing: 119.0
